In [1]:
import requests
import pandas as pd
from time import sleep
import os
import json

# Import API key from config file
from config import TMDB_API_KEY

# Get API key: from environment variable or config file
api_key = os.getenv("TMDB_API_KEY") or TMDB_API_KEY

# TMDB base URL
base_url = "https://api.themoviedb.org/3"

def fetch_movie_data(movie_id):
    """Fetch movie details from TMDB API."""
    url = f"{base_url}/movie/{movie_id}"
    params = {
        "api_key": api_key,
        "language": "en-US"
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for movie ID {movie_id}: {response.status_code}")
        try:
            print(response.json())  # Print error details from API
        except:
            pass
        return None

def main():
    """Main pipeline to fetch movies and build a DataFrame."""
    # Example movie IDs (test small batch first)
    movie_ids = [550, 551, 552, 553, 554]
    
    movie_data_list = []

    for movie_id in movie_ids:
        movie_data = fetch_movie_data(movie_id)
        if movie_data:
            print("\n🔍 API Response for Movie ID:", movie_id)
            print(json.dumps(movie_data, indent=4))  # Pretty print JSON for clarity
            
            movie_data_list.append(movie_data)
        sleep(1)  # Be respectful of rate limits

    # Convert to DataFrame
    if movie_data_list:
        df = pd.DataFrame(movie_data_list)

        print("\n Fetched Movie DataFrame:")
        print(df.head())

        print("\n DataFrame Columns:")
        print(df.columns)

        # Save DataFrame (optional)
        # df.to_csv("tmdb_movies_raw.csv", index=False)

    else:
        print("No movie data fetched.")

if __name__ == "__main__":
    main()




🔍 API Response for Movie ID: 550
{
    "adult": false,
    "backdrop_path": "/xRyINp9KfMLVjRiO5nCsoRDdvvF.jpg",
    "belongs_to_collection": null,
    "budget": 63000000,
    "genres": [
        {
            "id": 18,
            "name": "Drama"
        }
    ],
    "homepage": "http://www.foxmovies.com/movies/fight-club",
    "id": 550,
    "imdb_id": "tt0137523",
    "origin_country": [
        "US"
    ],
    "original_language": "en",
    "original_title": "Fight Club",
    "overview": "A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground \"fight clubs\" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.",
    "popularity": 30.9663,
    "poster_path": "/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg",
    "production_companies": [
        {
            "id": 711,
            "logo_path": "/tEiIH5QesdheJmDAqQwvtN60727.pn

In [2]:
url = f"https://api.themoviedb.org/3/discover/movie"
params = {
    "api_key": api_key,
    "language": "en-US",
    "primary_release_date.gte": "2000-01-01",
    "primary_release_date.lte": "2024-12-31",
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

print(f"Total movies matching query: {data['total_results']}")
print(f"Total pages: {data['total_pages']}")


Total movies matching query: 578672
Total pages: 28934


In [3]:
#  Storage for results
movie_data = []

# Loop through TMDB's /discover/movie endpoint
for page in range(1, 800):  # Start with 2 pages for quick testing; increase to 201 later
    discover_url = f"{base_url}/discover/movie"
    params = {
        "api_key": api_key,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "include_adult": False,
        "include_video": False,
        "page": page,
        "primary_release_date.gte": "2000-01-01",
        "primary_release_date.lte": "2024-12-31"
    }

    response = requests.get(discover_url, params=params)
    if response.status_code != 200:
        print(f"Failed on page {page} — Status: {response.status_code}")
        print(response.json())  # Show error
        break

    movies = response.json().get("results", [])

    for movie in movies:
        movie_id = movie["id"]

        # Fetch movie details
        movie_details = requests.get(f"{base_url}/movie/{movie_id}", params={"api_key": api_key}).json()
        credits = requests.get(f"{base_url}/movie/{movie_id}/credits", params={"api_key": api_key}).json()

        # Get director and top actor
        director = next((c["name"] for c in credits.get("crew", []) if c["job"] == "Director"), None)
        cast = [c["name"] for c in credits.get("cast", [])][:1]

        # Collect relevant fields
        movie_data.append({
            "title": movie_details.get("title"),
            "release_date": movie_details.get("release_date"),
            "budget": movie_details.get("budget"),
            "revenue": movie_details.get("revenue"),
            "runtime": movie_details.get("runtime"),
            "genres": [g["name"] for g in movie_details.get("genres", [])],
            "popularity": movie_details.get("popularity"),
            "vote_average": movie_details.get("vote_average"),
            "vote_count": movie_details.get("vote_count"),
            "original_language": movie_details.get("original_language"),
            "production_companies": [pc["name"] for pc in movie_details.get("production_companies", [])],
            "belongs_to_collection": movie_details.get("belongs_to_collection") is not None,
            "director": director,
            "lead_actor": cast[0] if cast else None
        })

    print(f"Page {page} complete — movies collected: {len(movie_data)}")
    sleep(0.3)

# Convert to DataFrame
df_movies = pd.DataFrame(movie_data)

# Print the DataFrame
df_movies # Preview first few rows


Page 1 complete — movies collected: 20
Page 2 complete — movies collected: 40
Page 3 complete — movies collected: 60
Page 4 complete — movies collected: 80
Page 5 complete — movies collected: 100
Page 6 complete — movies collected: 120
Page 7 complete — movies collected: 140
Page 8 complete — movies collected: 160
Page 9 complete — movies collected: 180
Page 10 complete — movies collected: 200
Page 11 complete — movies collected: 220
Page 12 complete — movies collected: 240
Page 13 complete — movies collected: 260
Page 14 complete — movies collected: 280
Page 15 complete — movies collected: 300
Page 16 complete — movies collected: 320
Page 17 complete — movies collected: 340
Page 18 complete — movies collected: 360
Page 19 complete — movies collected: 380
Page 20 complete — movies collected: 400
Page 21 complete — movies collected: 420
Page 22 complete — movies collected: 440
Page 23 complete — movies collected: 460
Page 24 complete — movies collected: 480
Page 25 complete — movies col

,title,release_date,budget,revenue,runtime,genres,popularity,vote_average,vote_count,original_language,production_companies,belongs_to_collection,director,lead_actor
0,Lilo & Stitch,2002-06-21,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,en,"[Walt Disney Pictures, Walt Disney Feature Ani...",True,Chris Sanders,Daveigh Chase
1,Panda Bear in Africa,2024-03-15,10000000,5514627,94,"[Animation, Adventure, Comedy, Family]",129.1975,6.447,38,nl,"[Cool Beans, A. Film Production, Comet Films, ...",False,Karsten Kiilerich,Yootha Wong-Loi-Sing
2,Brave Citizen,2023-10-25,0,2116112,112,"[Action, Drama, Comedy]",119.1160,6.807,44,ko,"[Studio N, Oscar 10 Studio, Vol Media]",False,Park Jin-pyo,Shin Hye-sun
3,Moana 2,2024-11-21,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,en,"[Walt Disney Pictures, Walt Disney Animation S...",True,David G. Derrick Jr.,Auliʻi Cravalho
4,The Great Escape,2023-05-26,200000,0,90,"[Action, Thriller]",100.8016,5.700,3,pt,[],False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Towa no Quon 2: Dancing Orchid in Chaos,2011-07-15,0,0,48,"[Action, Animation, Science Fiction, Fantasy]",2.0976,7.053,19,ja,"[BONES, Bandai Visual, Hakuhodo DY Media Partn...",True,Umanosuke Iida,Hiroshi Kamiya
9996,It's Complicated,2009-12-23,85000000,219103655,121,"[Comedy, Romance]",2.0976,6.500,1433,en,"[Universal Pictures, Relativity Media, Waverly...",False,Nancy Meyers,Meryl Streep
9997,Boxed In 2,2023-09-08,0,0,81,"[Drama, Romance]",2.0975,0.000,0,en,[],True,Daniel Stearns,Reginae Carter
9998,The House of the Serpent,2022-09-09,0,0,77,[Horror],2.0973,0.000,0,ja,[],False,Hiroshi Takahashi,Tomomi Kouno


In [4]:
df_movies = df_movies[(df_movies['budget'] != 0) & (df_movies['revenue'] != 0)]

df_movies['profit'] = df_movies['revenue'] - df_movies['budget']
df_movies['profit_margin'] = df_movies['profit'] / df_movies['revenue']
print("\nFinal DataFrame with Profit and Profit Margin:")
print(df_movies[['title', 'release_date', 'budget', 'revenue', 'profit', 'profit_margin']].head())

df_movies


Final DataFrame with Profit and Profit Margin:
                   title release_date     budget     revenue     profit  \
0          Lilo & Stitch   2002-06-21   80000000   273144151  193144151   
1   Panda Bear in Africa   2024-03-15   10000000     5514627   -4485373   
3                Moana 2   2024-11-21  150000000  1059544057  909544057   
6  Mufasa: The Lion King   2024-12-18  200000000   721046090  521046090   
8   Sonic the Hedgehog 3   2024-12-19  122000000   486018457  364018457   

   profit_margin  
0       0.707114  
1      -0.813359  
3       0.858430  
6       0.722625  
8       0.748981  


/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2387721848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['profit'] = df_movies['revenue'] - df_movies['budget']
/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2387721848.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['profit_margin'] = df_movies['profit'] / df_movies['revenue']


,title,release_date,budget,revenue,runtime,genres,popularity,vote_average,vote_count,original_language,production_companies,belongs_to_collection,director,lead_actor,profit,profit_margin
0,Lilo & Stitch,2002-06-21,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,en,"[Walt Disney Pictures, Walt Disney Feature Ani...",True,Chris Sanders,Daveigh Chase,193144151,0.707114
1,Panda Bear in Africa,2024-03-15,10000000,5514627,94,"[Animation, Adventure, Comedy, Family]",129.1975,6.447,38,nl,"[Cool Beans, A. Film Production, Comet Films, ...",False,Karsten Kiilerich,Yootha Wong-Loi-Sing,-4485373,-0.813359
3,Moana 2,2024-11-21,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,en,"[Walt Disney Pictures, Walt Disney Animation S...",True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430
6,Mufasa: The Lion King,2024-12-18,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,en,[Walt Disney Pictures],True,Barry Jenkins,Aaron Pierre,521046090,0.722625
8,Sonic the Hedgehog 3,2024-12-19,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,en,"[Paramount Pictures, Original Film, Marza Anim...",True,Jeff Fowler,Jim Carrey,364018457,0.748981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9964,W.,2008-10-17,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,en,"[Ixtlan Productions, Moritz Borman Productions]",False,Oliver Stone,Josh Brolin,4406464,0.149339
9967,Assassination of a High School President,2008-01-17,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,en,"[Yari Film Group, Vertigo Entertainment]",False,Brett Simon,Reece Thompson,-11430436,-164.315393
9978,The Yards,2000-04-27,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,en,"[Miramax, Industry Entertainment Partners]",False,James Gray,Mark Wahlberg,-23110648,-25.985940
9993,Drishyam,2013-12-19,750000,12000000,164,"[Thriller, Crime, Drama, Family]",2.0978,7.591,203,ml,[Aashirvad Cinemas],True,Jeethu Joseph,Mohanlal,11250000,0.937500


In [5]:
df_movies["release__year"] = pd.to_datetime(df_movies["release_date"], errors='coerce').dt.year

df_movies

/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/3566132683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies["release__year"] = pd.to_datetime(df_movies["release_date"], errors='coerce').dt.year


,title,release_date,budget,revenue,runtime,genres,popularity,vote_average,vote_count,original_language,production_companies,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year
0,Lilo & Stitch,2002-06-21,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,en,"[Walt Disney Pictures, Walt Disney Feature Ani...",True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002
1,Panda Bear in Africa,2024-03-15,10000000,5514627,94,"[Animation, Adventure, Comedy, Family]",129.1975,6.447,38,nl,"[Cool Beans, A. Film Production, Comet Films, ...",False,Karsten Kiilerich,Yootha Wong-Loi-Sing,-4485373,-0.813359,2024
3,Moana 2,2024-11-21,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,en,"[Walt Disney Pictures, Walt Disney Animation S...",True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024
6,Mufasa: The Lion King,2024-12-18,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,en,[Walt Disney Pictures],True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024
8,Sonic the Hedgehog 3,2024-12-19,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,en,"[Paramount Pictures, Original Film, Marza Anim...",True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9964,W.,2008-10-17,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,en,"[Ixtlan Productions, Moritz Borman Productions]",False,Oliver Stone,Josh Brolin,4406464,0.149339,2008
9967,Assassination of a High School President,2008-01-17,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,en,"[Yari Film Group, Vertigo Entertainment]",False,Brett Simon,Reece Thompson,-11430436,-164.315393,2008
9978,The Yards,2000-04-27,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,en,"[Miramax, Industry Entertainment Partners]",False,James Gray,Mark Wahlberg,-23110648,-25.985940,2000
9993,Drishyam,2013-12-19,750000,12000000,164,"[Thriller, Crime, Drama, Family]",2.0978,7.591,203,ml,[Aashirvad Cinemas],True,Jeethu Joseph,Mohanlal,11250000,0.937500,2013


In [6]:
df_movies = df_movies[df_movies["original_language"] == "en"]

df_movies

,title,release_date,budget,revenue,runtime,genres,popularity,vote_average,vote_count,original_language,production_companies,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year
0,Lilo & Stitch,2002-06-21,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,en,"[Walt Disney Pictures, Walt Disney Feature Ani...",True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002
3,Moana 2,2024-11-21,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,en,"[Walt Disney Pictures, Walt Disney Animation S...",True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024
6,Mufasa: The Lion King,2024-12-18,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,en,[Walt Disney Pictures],True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024
8,Sonic the Hedgehog 3,2024-12-19,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,en,"[Paramount Pictures, Original Film, Marza Anim...",True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024
9,Mission: Impossible - Dead Reckoning Part One,2023-07-08,291000000,571125435,164,"[Action, Adventure, Thriller]",73.4752,7.532,4311,en,"[Paramount Pictures, Skydance Media, TC Produc...",True,Christopher McQuarrie,Tom Cruise,280125435,0.490480,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,The Curse of the Jade Scorpion,2001-08-05,33000000,18914307,103,"[Comedy, Crime, Romance]",2.1033,6.601,786,en,"[Gravier Productions, DreamWorks Pictures, VCL...",False,Woody Allen,Dan Aykroyd,-14085693,-0.744711,2001
9964,W.,2008-10-17,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,en,"[Ixtlan Productions, Moritz Borman Productions]",False,Oliver Stone,Josh Brolin,4406464,0.149339,2008
9967,Assassination of a High School President,2008-01-17,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,en,"[Yari Film Group, Vertigo Entertainment]",False,Brett Simon,Reece Thompson,-11430436,-164.315393,2008
9978,The Yards,2000-04-27,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,en,"[Miramax, Industry Entertainment Partners]",False,James Gray,Mark Wahlberg,-23110648,-25.985940,2000


In [7]:
df_movies['production_company'] = df_movies['production_companies'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

df_movies

/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2256211837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['production_company'] = df_movies['production_companies'].apply(


,title,release_date,budget,revenue,runtime,genres,popularity,vote_average,vote_count,original_language,production_companies,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year,production_company
0,Lilo & Stitch,2002-06-21,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,en,"[Walt Disney Pictures, Walt Disney Feature Ani...",True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002,Walt Disney Pictures
3,Moana 2,2024-11-21,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,en,"[Walt Disney Pictures, Walt Disney Animation S...",True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024,Walt Disney Pictures
6,Mufasa: The Lion King,2024-12-18,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,en,[Walt Disney Pictures],True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024,Walt Disney Pictures
8,Sonic the Hedgehog 3,2024-12-19,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,en,"[Paramount Pictures, Original Film, Marza Anim...",True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024,Paramount Pictures
9,Mission: Impossible - Dead Reckoning Part One,2023-07-08,291000000,571125435,164,"[Action, Adventure, Thriller]",73.4752,7.532,4311,en,"[Paramount Pictures, Skydance Media, TC Produc...",True,Christopher McQuarrie,Tom Cruise,280125435,0.490480,2023,Paramount Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,The Curse of the Jade Scorpion,2001-08-05,33000000,18914307,103,"[Comedy, Crime, Romance]",2.1033,6.601,786,en,"[Gravier Productions, DreamWorks Pictures, VCL...",False,Woody Allen,Dan Aykroyd,-14085693,-0.744711,2001,Gravier Productions
9964,W.,2008-10-17,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,en,"[Ixtlan Productions, Moritz Borman Productions]",False,Oliver Stone,Josh Brolin,4406464,0.149339,2008,Ixtlan Productions
9967,Assassination of a High School President,2008-01-17,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,en,"[Yari Film Group, Vertigo Entertainment]",False,Brett Simon,Reece Thompson,-11430436,-164.315393,2008,Yari Film Group
9978,The Yards,2000-04-27,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,en,"[Miramax, Industry Entertainment Partners]",False,James Gray,Mark Wahlberg,-23110648,-25.985940,2000,Miramax


In [8]:
df_movies.drop(columns=['release_date', 'original_language', 'production_companies'], inplace=True)

df_movies

/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/1524026046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies.drop(columns=['release_date', 'original_language', 'production_companies'], inplace=True)


,title,budget,revenue,runtime,genres,popularity,vote_average,vote_count,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year,production_company
0,Lilo & Stitch,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002,Walt Disney Pictures
3,Moana 2,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024,Walt Disney Pictures
6,Mufasa: The Lion King,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024,Walt Disney Pictures
8,Sonic the Hedgehog 3,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024,Paramount Pictures
9,Mission: Impossible - Dead Reckoning Part One,291000000,571125435,164,"[Action, Adventure, Thriller]",73.4752,7.532,4311,True,Christopher McQuarrie,Tom Cruise,280125435,0.490480,2023,Paramount Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,The Curse of the Jade Scorpion,33000000,18914307,103,"[Comedy, Crime, Romance]",2.1033,6.601,786,False,Woody Allen,Dan Aykroyd,-14085693,-0.744711,2001,Gravier Productions
9964,W.,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,False,Oliver Stone,Josh Brolin,4406464,0.149339,2008,Ixtlan Productions
9967,Assassination of a High School President,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,False,Brett Simon,Reece Thompson,-11430436,-164.315393,2008,Yari Film Group
9978,The Yards,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,False,James Gray,Mark Wahlberg,-23110648,-25.985940,2000,Miramax


In [9]:
df_movies.reset_index(drop=True, inplace=True)

df_movies

,title,budget,revenue,runtime,genres,popularity,vote_average,vote_count,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year,production_company
0,Lilo & Stitch,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002,Walt Disney Pictures
1,Moana 2,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024,Walt Disney Pictures
2,Mufasa: The Lion King,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024,Walt Disney Pictures
3,Sonic the Hedgehog 3,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024,Paramount Pictures
4,Mission: Impossible - Dead Reckoning Part One,291000000,571125435,164,"[Action, Adventure, Thriller]",73.4752,7.532,4311,True,Christopher McQuarrie,Tom Cruise,280125435,0.490480,2023,Paramount Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516,The Curse of the Jade Scorpion,33000000,18914307,103,"[Comedy, Crime, Romance]",2.1033,6.601,786,False,Woody Allen,Dan Aykroyd,-14085693,-0.744711,2001,Gravier Productions
3517,W.,25100000,29506464,129,"[Drama, History]",2.1015,6.224,642,False,Oliver Stone,Josh Brolin,4406464,0.149339,2008,Ixtlan Productions
3518,Assassination of a High School President,11500000,69564,93,"[Comedy, Crime]",2.1011,5.700,307,False,Brett Simon,Reece Thompson,-11430436,-164.315393,2008,Yari Film Group
3519,The Yards,24000000,889352,115,"[Crime, Drama, Thriller]",2.1001,6.200,465,False,James Gray,Mark Wahlberg,-23110648,-25.985940,2000,Miramax


In [10]:
# Function to extract up to 3 genres
def extract_genres(genre_list):
    # Pad with None if fewer than 3 genres
    genres = genre_list[:3] + [None] * (3 - len(genre_list))
    return pd.Series(genres, index=["genre1", "genre2", "genre3"])
# Apply the function
df_movies[["genre1", "genre2", "genre3"]] = df_movies["genres"].apply(extract_genres)
# Optional: drop the original genres column if no longer needed
# df_movies.drop(columns=["genres"], inplace=True)
# Preview the result
df_movies.head()

/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2624932210.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies[["genre1", "genre2", "genre3"]] = df_movies["genres"].apply(extract_genres)
/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2624932210.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies[["genre1", "genre2", "genre3"]] = df_movies["genres"].apply(extract_genres)
/var/folders/3n/t0775bfj4931hr98hy7197340000gn/T/ipykernel_5216/2624932210.py:7: Setti

,title,budget,revenue,runtime,genres,popularity,vote_average,vote_count,belongs_to_collection,director,lead_actor,profit,profit_margin,release__year,production_company,genre1,genre2,genre3
0,Lilo & Stitch,80000000,273144151,85,"[Animation, Family, Comedy]",182.4251,7.547,6556,True,Chris Sanders,Daveigh Chase,193144151,0.707114,2002,Walt Disney Pictures,Animation,Family,Comedy
1,Moana 2,150000000,1059544057,100,"[Animation, Adventure, Family, Comedy]",114.2420,7.081,2418,True,David G. Derrick Jr.,Auliʻi Cravalho,909544057,0.858430,2024,Walt Disney Pictures,Animation,Adventure,Family
2,Mufasa: The Lion King,200000000,721046090,118,"[Adventure, Family, Animation]",85.5401,7.397,2165,True,Barry Jenkins,Aaron Pierre,521046090,0.722625,2024,Walt Disney Pictures,Adventure,Family,Animation
3,Sonic the Hedgehog 3,122000000,486018457,110,"[Action, Science Fiction, Comedy, Family]",81.6122,7.715,2633,True,Jeff Fowler,Jim Carrey,364018457,0.748981,2024,Paramount Pictures,Action,Science Fiction,Comedy
4,Mission: Impossible - Dead Reckoning Part One,291000000,571125435,164,"[Action, Adventure, Thriller]",73.4752,7.532,4311,True,Christopher McQuarrie,Tom Cruise,280125435,0.490480,2023,Paramount Pictures,Action,Adventure,Thriller


In [11]:
# Export the final DataFrame to a CSV file
output_file = "tmdb_movies_500_page_final.csv"
df_movies.to_csv(output_file, index=False)
print(f"\nData exported to {output_file}")


Data exported to tmdb_movies_500_page_final.csv
